In [1]:
import csv
import datetime as dt
from psaw import PushshiftAPI

In [2]:
outfile = 'reddit_submissions_casual_conv.csv'

api = PushshiftAPI()

start_epoch=int(dt.datetime(2020, 1, 1).timestamp())
end_epoch=int(dt.datetime(2020, 4, 30).timestamp())

data = api.search_submissions(after=start_epoch, before=end_epoch,
                            subreddit='casualconversation',
                            filter=['url','id', 'author', 'title', 'selftext', 'score', 'num_comments', 'upvote_ratio', 'subreddit_subscribers'],
                            limit=100000000)

In [2]:
# Create .csv with column headers
with open(outfile, 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['date', 'id', 'author', 'url', 'score', 'upvote_ratio', 'num_comments', 'subreddit_subscribers', 'title', 'text'])

In [ ]:
counter = 0
with open(outfile, 'a', newline='', encoding='utf8') as file:
    writer = csv.writer(file)
    for sub in data:
        try:
            if sub.d_['author'] != '[deleted]':
                query = sub.d_['selftext']
                len_text = sub.d_['selftext']
                if (query == '[removed]') or (len(len_text) <= 500):
                    pass
                else:
                    counter += 1
                    if counter % 500 == 0:
                        print('Results found',counter)
                    text = sub.d_['selftext'].encode('ascii', 'ignore').decode().replace('\n', '')
                    date = dt.datetime.fromtimestamp(sub.d_['created_utc']).strftime('%Y-%m-%d %H:%M:%S')
                    try:
                        writer.writerow([date, sub.d_['id'], sub.d_['author'], sub.d_['url'], sub.d_['score'], sub.d_['upvote_ratio'], sub.d_['num_comments'], sub.d_['subreddit_subscribers'], sub.d_['title'], text])
                    except Exception:
                        writer.writerow([date, sub.d_['id'], sub.d_['author'], sub.d_['url'], sub.d_['score'], 0.0, sub.d_['num_comments'], sub.d_['subreddit_subscribers'], sub.d_['title'], text])
        except Exception as e:
            print('Error at count', counter, e)
            print(sub.d_)
print('done!')